In [5]:
from google.colab import files
uploaded = files.upload()


Saving Product Dataset.zip to Product Dataset.zip


In [6]:
ZIP_PATH = "/content/Product Dataset.zip"
EXTRACT_DIR = "/content/product_data"

import zipfile, os
os.makedirs(EXTRACT_DIR, exist_ok=True)

with zipfile.ZipFile(ZIP_PATH, 'r') as z:
    z.extractall(EXTRACT_DIR)

print("Extraction done")


Extraction done


In [7]:
!ls /content


 product_data  'Product Dataset.zip'   sample_data


In [8]:
import os, glob, pandas as pd

BASE = "/content/product_data"
csv_candidates = glob.glob(BASE + "/**/*.csv", recursive=True)

print("Found CSVs:")
for p in csv_candidates[:20]:
    print(p)

# pick the main file (your dataset name)
target = [p for p in csv_candidates if "Message Group - Product.csv" in os.path.basename(p)]
csv_path = target[0] if target else csv_candidates[0]

print("\nUsing:", csv_path)
df = pd.read_csv(csv_path)
print("Shape:", df.shape)
df.head()


Found CSVs:
/content/product_data/Product Dataset/Message Group - Product.csv

Using: /content/product_data/Product Dataset/Message Group - Product.csv
Shape: (4566, 11)


,S.No,BrandName,Product ID,Product Name,Brand Desc,Product Size,Currancy,MRP,SellPrice,Discount,Category
0,1,4711,FR001,Cologne Fragrance,ekw eau de cologne 400 ml,Small,Rs.,3900,3120,20% off,Fragrance-Women
1,2,109f,DRW1,DRW1 - Westernwear-Women,womens v- neck short dress - yellow,"Size:Medium,Small,X-Large,XX-Large",Rs.,1899,569,70% off,Westernwear-Women
2,3,109f,DRW2,DRW2 - Westernwear-Women,womens round neck solid top - black,"Size:Large,Medium,Small,X-Large",Rs.,1499,599,60% off,Westernwear-Women
3,4,109f,DRW3,DRW3 - Westernwear-Women,womens round neck stripe shift dress - red,"Size:Medium,Small",Rs.,1599,639,60% off,Westernwear-Women
4,5,109f,DRW4,DRW4 - Westernwear-Women,womens round neck solid high low top - black,"Size:Large,Medium,Small,X-Large",Rs.,1199,479,60% off,Westernwear-Women


In [9]:
import numpy as np

df = df.copy()

# Ensure columns exist
cols = ["BrandName","Product ID","Product Name","Brand Desc","Product Size","Currancy","Discount","Category","MRP","SellPrice"]
for c in cols:
    if c not in df.columns:
        df[c] = ""

# Clean strings
for c in ["BrandName","Product ID","Product Name","Brand Desc","Product Size","Currancy","Discount","Category"]:
    df[c] = df[c].astype(str).fillna("").str.strip()

def to_float(x):
    try:
        return float(str(x).replace(",","").strip())
    except:
        return np.nan

df["MRP_num"] = df["MRP"].apply(to_float)
df["SellPrice_num"] = df["SellPrice"].apply(to_float)

df["text"] = (
    "brand: " + df["BrandName"] + " | "
    "name: " + df["Product Name"] + " | "
    "desc: " + df["Brand Desc"] + " | "
    "category: " + df["Category"] + " | "
    "size: " + df["Product Size"] + " | "
    "price: " + df["SellPrice"].astype(str)
)

df[["Product ID","Product Name","Category","text"]].head()


,Product ID,Product Name,Category,text
0,FR001,Cologne Fragrance,Fragrance-Women,brand: 4711 | name: Cologne Fragrance | desc: ...
1,DRW1,DRW1 - Westernwear-Women,Westernwear-Women,brand: 109f | name: DRW1 - Westernwear-Women |...
2,DRW2,DRW2 - Westernwear-Women,Westernwear-Women,brand: 109f | name: DRW2 - Westernwear-Women |...
3,DRW3,DRW3 - Westernwear-Women,Westernwear-Women,brand: 109f | name: DRW3 - Westernwear-Women |...
4,DRW4,DRW4 - Westernwear-Women,Westernwear-Women,brand: 109f | name: DRW4 - Westernwear-Women |...


In [10]:
!pip -q install faiss-cpu sentence-transformers

import faiss
from sentence_transformers import SentenceTransformer

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)

texts = df["text"].tolist()
emb = embedder.encode(texts, batch_size=128, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

dim = emb.shape[1]
index = faiss.IndexFlatIP(dim)  # inner product works as cosine when normalized
index.add(emb.astype(np.float32))

id_to_idx = {pid: i for i, pid in enumerate(df["Product ID"].tolist()) if str(pid).strip() != ""}
print("Embeddings:", emb.shape, "| FAISS size:", index.ntotal)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/36 [00:00<?, ?it/s]

Embeddings: (4566, 384) | FAISS size: 4566


In [11]:
def search_by_text(query, top_k=10):
    qemb = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
    scores, idxs = index.search(qemb, top_k)
    out = []
    for j, s in zip(idxs[0].tolist(), scores[0].tolist()):
        r = df.iloc[j]
        out.append((round(float(s), 4), r["Product ID"], r["Product Name"], r["Category"], r["BrandName"]))
    return out

search_by_text("men black tshirt", top_k=10)[:5]


[(0.3994, 'AND608', 'AND608 - Westernwear-Women', 'Westernwear-Women', 'and'),
 (0.3923,
  'COVER STORY87',
  'COVER STORY87 - Westernwear-Women',
  'Westernwear-Women',
  'cover story'),
 (0.3866,
  'COVER STORY132',
  'COVER STORY132 - Westernwear-Women',
  'Westernwear-Women',
  'cover story'),
 (0.3788, 'AND485', 'AND485 - Westernwear-Women', 'Westernwear-Women', 'and'),
 (0.3781,
  'ADIDAS8',
  'ADIDAS8 - Westernwear-Women',
  'Westernwear-Women',
  'adidas')]

In [12]:
from collections import defaultdict
import math

cat_to_indices = defaultdict(list)
cats = df["Category"].tolist()
for i, c in enumerate(cats):
    cat_to_indices[c].append(i)

relevant = {}
for i, c in enumerate(cats):
    s = set(cat_to_indices[c])
    s.discard(i)
    relevant[i] = s

def search_by_idx(i, top_k=20):
    q = emb[i].astype(np.float32).reshape(1, -1)
    scores, idxs = index.search(q, top_k + 1)
    out = []
    for j, s in zip(idxs[0].tolist(), scores[0].tolist()):
        if j == i:
            continue
        out.append((j, s))
        if len(out) >= top_k:
            break
    return out

def precision_at_k(retrieved, rel_set, k):
    r = retrieved[:k]
    return sum(1 for x in r if x in rel_set) / k

def recall_at_k(retrieved, rel_set, k):
    return (sum(1 for x in retrieved[:k] if x in rel_set) / len(rel_set)) if len(rel_set) else np.nan

def map_at_k(retrieved, rel_set, k):
    hits, ap = 0, 0.0
    for rank, idx in enumerate(retrieved[:k], start=1):
        if idx in rel_set:
            hits += 1
            ap += hits / rank
    return ap / min(len(rel_set), k) if len(rel_set) else np.nan

def mrr_at_k(retrieved, rel_set, k):
    for rank, idx in enumerate(retrieved[:k], start=1):
        if idx in rel_set:
            return 1.0 / rank
    return 0.0

def ndcg_at_k(retrieved, rel_set, k):
    dcg = 0.0
    for rank, idx in enumerate(retrieved[:k], start=1):
        rel_i = 1.0 if idx in rel_set else 0.0
        dcg += (2**rel_i - 1) / math.log2(rank + 1)

    ideal = min(len(rel_set), k)
    if ideal == 0:
        return np.nan
    idcg = sum((2**1 - 1) / math.log2(r + 1) for r in range(2, ideal + 2))
    return dcg / idcg

def evaluate(k_list=[5,10,20], sample_n=1500, seed=42):
    rng = np.random.default_rng(seed)
    all_idx = np.arange(len(df))
    if sample_n and sample_n < len(df):
        all_idx = rng.choice(all_idx, size=sample_n, replace=False)

    rows = []
    maxk = max(k_list)
    for k in k_list:
        P=R=MAP=MRR=NDCG=[]
        P, R, MAP, MRR, NDCG = [], [], [], [], []
        count = 0
        for i in all_idx:
            rel_set = relevant[i]
            if len(rel_set) == 0:
                continue
            retrieved = [j for j,_ in search_by_idx(i, top_k=maxk)]
            P.append(precision_at_k(retrieved, rel_set, k))
            R.append(recall_at_k(retrieved, rel_set, k))
            MAP.append(map_at_k(retrieved, rel_set, k))
            MRR.append(mrr_at_k(retrieved, rel_set, k))
            NDCG.append(ndcg_at_k(retrieved, rel_set, k))
            count += 1

        rows.append({
            "K": k,
            "Precision@K": float(np.nanmean(P)),
            "Recall@K": float(np.nanmean(R)),
            "MAP@K": float(np.nanmean(MAP)),
            "MRR@K": float(np.nanmean(MRR)),
            "nDCG@K": float(np.nanmean(NDCG)),
            "Evaluated Queries": count
        })
    return pd.DataFrame(rows)

metrics_df = evaluate([5,10,20], sample_n=1500)
metrics_df


,K,Precision@K,Recall@K,MAP@K,MRR@K,nDCG@K,Evaluated Queries
0,5,0.996933,0.007739,0.996742,0.998,1.275715,1500
1,10,0.996267,0.015469,0.996025,0.998,1.184611,1500
2,20,0.995800,0.030929,0.995519,0.998,1.119516,1500


In [13]:
import os, pickle

ASSETS = "/content/app_assets"
os.makedirs(ASSETS, exist_ok=True)

faiss.write_index(index, os.path.join(ASSETS, "faiss.index"))
np.save(os.path.join(ASSETS, "emb.npy"), emb.astype(np.float32))
df.to_csv(os.path.join(ASSETS, "products.csv"), index=False)
metrics_df.to_csv(os.path.join(ASSETS, "metrics.csv"), index=False)

with open(os.path.join(ASSETS, "id_to_idx.pkl"), "wb") as f:
    pickle.dump(id_to_idx, f)

print("Saved:", ASSETS)


Saved: /content/app_assets


In [17]:
%%writefile /content/app.py
import os, pickle
import numpy as np
import pandas as pd
import faiss
import streamlit as st
from sentence_transformers import SentenceTransformer

ASSETS = "/content/app_assets"

@st.cache_resource
def load_all():
    df = pd.read_csv(os.path.join(ASSETS, "products.csv"))
    emb = np.load(os.path.join(ASSETS, "emb.npy"))
    index = faiss.read_index(os.path.join(ASSETS, "faiss.index"))
    with open(os.path.join(ASSETS, "id_to_idx.pkl"), "rb") as f:
        id_to_idx = pickle.load(f)
    metrics = pd.read_csv(os.path.join(ASSETS, "metrics.csv"))
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    return df, emb, index, id_to_idx, metrics, model

df, emb, index, id_to_idx, metrics, model = load_all()

st.title("Product Matching Application (FAISS)")
st.dataframe(metrics, use_container_width=True)

st.divider()
mode = st.radio("Search Mode", ["By Product ID", "By Free Text"], horizontal=True)
top_k = st.slider("Top K", 3, 30, 10)

def search_by_idx(i, k):
    q = emb[i].astype(np.float32).reshape(1, -1)
    scores, idxs = index.search(q, k+1)
    out = []
    for j, s in zip(idxs[0].tolist(), scores[0].tolist()):
        if j == i:
            continue
        out.append((j, float(s)))
        if len(out) >= k:
            break
    return out

def search_by_text(query, k):
    qemb = model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
    scores, idxs = index.search(qemb, k)
    return list(zip(idxs[0].tolist(), scores[0].tolist()))

def table_from_hits(hits):
    rows = []
    for j, s in hits:
        r = df.iloc[j]
        rows.append({
            "score": round(float(s), 4),
            "Product ID": r.get("Product ID",""),
            "Product Name": r.get("Product Name",""),
            "Category": r.get("Category",""),
            "BrandName": r.get("BrandName",""),
        })
    return pd.DataFrame(rows)

if mode == "By Product ID":
    pid = st.text_input("Product ID (exact)", "")
    if pid:
        pid = pid.strip()
        if pid not in id_to_idx:
            st.error("Product ID not found.")
        else:
            i = id_to_idx[pid]
            st.subheader("Selected Product")
            st.write(df.iloc[i][["Product ID","Product Name","Category","BrandName","Brand Desc","Product Size","MRP","SellPrice","Discount"]])
            st.subheader("Top Matches")
            hits = search_by_idx(i, top_k)
            st.dataframe(table_from_hits(hits), use_container_width=True)
else:
    q = st.text_input("Query", "black t-shirt for men")
    if q:
        st.subheader("Top Matches")
        hits = search_by_text(q, top_k)
        st.dataframe(table_from_hits(hits), use_container_width=True)


Overwriting /content/app.py


In [18]:
!pip -q install streamlit pyngrok

from pyngrok import ngrok
import subprocess

proc = subprocess.Popen(["streamlit", "run", "/content/app.py", "--server.port", "8501", "--server.address", "0.0.0.0"])
public_url = ngrok.connect(8501)
print("✅ Open this Streamlit URL:", public_url)


✅ Open this Streamlit URL: NgrokTunnel: "https://pseudonymously-matrilateral-jenee.ngrok-free.dev" -> "http://localhost:8501"


In [16]:
from pyngrok import ngrok

ngrok.set_auth_token("34uniVwpJPKje9EQ4yuFWASb77w_6YSsryqoQgWPwirmbao9E")  # paste only inside your Colab cell
public_url = ngrok.connect(8501)
print("✅ Streamlit URL:", public_url)


✅ Streamlit URL: NgrokTunnel: "https://pseudonymously-matrilateral-jenee.ngrok-free.dev" -> "http://localhost:8501"


In [19]:
!pip -q install streamlit rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.3 MB/s eta 0:00:00


In [25]:
%%writefile /content/app.py
import os, pickle
import numpy as np
import pandas as pd
import faiss
import streamlit as st
from sentence_transformers import SentenceTransformer
from rapidfuzz import fuzz

ASSETS = "/content/app_assets"

# ----------------------------
# Helpers
# ----------------------------
def safe_str(x):
    if pd.isna(x):
        return ""
    return str(x).strip()

def to_float(x):
    try:
        return float(str(x).replace(",", "").strip())
    except:
        return np.nan

def parse_discount(x):
    x = safe_str(x).replace("%", "")
    return to_float(x)

def lexical_score(query: str, lex_text: str) -> float:
    # fuzzy ratio 0..100 -> 0..1
    return fuzz.token_set_ratio(query.lower(), (lex_text or "").lower()) / 100.0

def product_card(df: pd.DataFrame, idx: int, title="Product"):
    r = df.iloc[idx]
    st.markdown(f"#### {title}")
    st.caption(
        f"ID: **{safe_str(r.get('Product ID',''))}**  •  "
        f"Category: **{safe_str(r.get('Category',''))}**  •  "
        f"Brand: **{safe_str(r.get('BrandName',''))}**"
    )
    st.write({
        "Product Name": safe_str(r.get("Product Name","")),
        "Brand Desc": safe_str(r.get("Brand Desc","")),
        "Product Size": safe_str(r.get("Product Size","")),
        "MRP": safe_str(r.get("MRP","")),
        "SellPrice": safe_str(r.get("SellPrice","")),
        "Discount": safe_str(r.get("Discount","")),
        "Currency": safe_str(r.get("Currancy","")),
    })

def build_display_table(df: pd.DataFrame, hits):
    """
    hits: list of tuples (idx, semantic, lexical, final)
    """
    rows = []
    for j, sem, lex, final in hits:
        r = df.iloc[j]
        rows.append({
            "final_score": round(float(final), 4),
            "semantic": round(float(sem), 4),
            "lexical": round(float(lex), 4),
            "Product ID": r.get("Product ID",""),
            "Product Name": r.get("Product Name",""),
            "Category": r.get("Category",""),
            "BrandName": r.get("BrandName",""),
            "SellPrice": r.get("SellPrice",""),
            "MRP": r.get("MRP",""),
            "Discount": r.get("Discount",""),
        })
    return pd.DataFrame(rows)

# ----------------------------
# Load assets
# ----------------------------
@st.cache_resource
def load_all():
    df = pd.read_csv(os.path.join(ASSETS, "products.csv"))
    emb = np.load(os.path.join(ASSETS, "emb.npy"))
    index = faiss.read_index(os.path.join(ASSETS, "faiss.index"))
    mpath = os.path.join(ASSETS, "metrics.csv")
    metrics = pd.read_csv(mpath) if os.path.exists(mpath) else None

    # Note: id_to_idx.pkl might be outdated if Product ID formatting changes,
    # so we will rebuild id_to_idx after normalizing Product ID.
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    return df, emb, index, metrics, model

df, emb, index, metrics, model = load_all()

# ----------------------------
# Normalize IDs (CRITICAL FIX)
# ----------------------------
if "Product ID" not in df.columns:
    st.error("Column 'Product ID' not found in dataset.")
    st.stop()

df["Product ID"] = (
    df["Product ID"]
    .astype(str)
    .str.replace(r"\.0$", "", regex=True)
    .str.strip()
)

# Rebuild lookup using normalized Product ID
id_to_idx = {
    pid: i for i, pid in enumerate(df["Product ID"].tolist())
    if pid and pid.lower() != "nan"
}

# ----------------------------
# Prepare numeric & lexical fields
# ----------------------------
for c in ["SellPrice", "MRP", "Discount"]:
    if c not in df.columns:
        df[c] = ""

df["SellPrice_num"] = df["SellPrice"].apply(to_float)
df["MRP_num"] = df["MRP"].apply(to_float)
df["Discount_num"] = df["Discount"].apply(parse_discount)

lex_fields = ["Product Name", "BrandName", "Brand Desc", "Category", "Product Size"]
for c in lex_fields:
    if c not in df.columns:
        df[c] = ""

df["lex_text"] = (
    df["Product Name"].astype(str) + " " +
    df["BrandName"].astype(str) + " " +
    df["Brand Desc"].astype(str) + " " +
    df["Category"].astype(str) + " " +
    df["Product Size"].astype(str)
).str.lower()

# ----------------------------
# Page config + style
# ----------------------------
st.set_page_config(page_title="Product Matching (FAISS)", layout="wide")

st.markdown(
    """
    <style>
      .block-container { padding-top: 1.2rem; padding-bottom: 2rem; }
      div[data-testid="stMetric"] { background: rgba(255,255,255,0.03); padding: 12px; border-radius: 12px; }
      .stTabs [data-baseweb="tab-list"] { gap: 10px; }
      .stTabs [data-baseweb="tab"] { padding: 10px 14px; border-radius: 12px; }
    </style>
    """,
    unsafe_allow_html=True
)

st.title("🧩 Product Matching Dashboard (FAISS + Embeddings)")
st.caption("Semantic search with FAISS + optional hybrid re-ranking for stronger product match quality.")

# ----------------------------
# Sidebar controls
# ----------------------------
st.sidebar.header("Controls")

mode = st.sidebar.radio("Search mode", ["By Product ID", "By Free Text"], index=1)
top_k = st.sidebar.slider("Top K results", 5, 50, 15)

use_hybrid = st.sidebar.toggle("Hybrid re-ranking (semantic + keyword)", value=True)
hybrid_weight = st.sidebar.slider("Hybrid weight (semantic)", 0.0, 1.0, 0.75, 0.05, disabled=not use_hybrid)

min_score = st.sidebar.slider("Min similarity threshold", 0.0, 1.0, 0.00, 0.01)

st.sidebar.markdown("---")
st.sidebar.subheader("Filters")

all_cats = sorted([c for c in df.get("Category", pd.Series([], dtype=str)).astype(str).unique().tolist() if c.strip() != ""])
all_brands = sorted([b for b in df.get("BrandName", pd.Series([], dtype=str)).astype(str).unique().tolist() if b.strip() != ""])

cats = st.sidebar.multiselect("Category", options=all_cats, default=[])
brands = st.sidebar.multiselect("Brand", options=all_brands, default=[])

# Price range sliders (safe)
sp_vals = df["SellPrice_num"].values
sp_min = float(np.nanmin(sp_vals)) if np.isfinite(np.nanmin(sp_vals)) else 0.0
sp_max = float(np.nanmax(sp_vals)) if np.isfinite(np.nanmax(sp_vals)) else 0.0
price_range = st.sidebar.slider("Sell price range", float(sp_min), float(sp_max), (float(sp_min), float(sp_max))) if sp_max > sp_min else (sp_min, sp_max)

d_vals = df["Discount_num"].values
d_min = float(np.nanmin(d_vals)) if np.isfinite(np.nanmin(d_vals)) else 0.0
d_max = float(np.nanmax(d_vals)) if np.isfinite(np.nanmax(d_vals)) else 0.0
disc_range = st.sidebar.slider("Discount range", float(d_min), float(d_max), (float(d_min), float(d_max))) if d_max > d_min else (d_min, d_max)

# ----------------------------
# Apply filters (CRITICAL FIX: allow missing numeric values)
# ----------------------------
mask = pd.Series(True, index=df.index)

if cats:
    mask &= df["Category"].astype(str).isin(cats)
if brands:
    mask &= df["BrandName"].astype(str).isin(brands)

price_ok = df["SellPrice_num"].isna() | df["SellPrice_num"].between(price_range[0], price_range[1])
disc_ok  = df["Discount_num"].isna() | df["Discount_num"].between(disc_range[0], disc_range[1])

mask &= price_ok
mask &= disc_ok

allowed_idx = np.where(mask.values)[0]
allowed_set = set(allowed_idx.tolist())
st.sidebar.caption(f"Allowed after filters: {len(allowed_idx)} / {len(df)}")

# ----------------------------
# Search functions
# ----------------------------
def faiss_search_by_vector(qvec, k):
    scores, idxs = index.search(qvec.astype(np.float32), k)
    return idxs[0].tolist(), scores[0].tolist()

def post_filter_and_rerank(query, idxs, scores, exclude_idx=None):
    hits = []
    for j, s in zip(idxs, scores):
        j = int(j)
        if exclude_idx is not None and j == exclude_idx:
            continue
        if j not in allowed_set:
            continue
        sem = float(s)
        if sem < float(min_score):
            continue

        lex = lexical_score(query, df.at[j, "lex_text"]) if use_hybrid else 0.0
        final = (hybrid_weight * sem + (1.0 - hybrid_weight) * lex) if use_hybrid else sem
        hits.append((j, sem, lex, final))

    hits.sort(key=lambda x: x[3], reverse=True)
    return hits[:top_k]

def search_by_text(query):
    qemb = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    idxs, scores = faiss_search_by_vector(qemb, k=max(200, top_k * 20))
    hits = post_filter_and_rerank(query, idxs, scores)

    # Professional fallback: if filters/threshold too strict, show raw FAISS
    if len(hits) == 0:
        st.warning("No matches found with current filters/threshold. Showing raw FAISS matches (auto-fallback).")
        idxs2, scores2 = faiss_search_by_vector(qemb, k=top_k)
        hits = [(int(j), float(s), 0.0, float(s)) for j, s in zip(idxs2, scores2)]
    return hits

def search_by_product_idx(i):
    query = (safe_str(df.at[i, "Product Name"]) + " " + safe_str(df.at[i, "BrandName"])).strip()
    qemb = emb[i].reshape(1, -1)
    idxs, scores = faiss_search_by_vector(qemb, k=max(200, top_k * 20))
    hits = post_filter_and_rerank(query, idxs, scores, exclude_idx=i)

    if len(hits) == 0:
        st.warning("No matches found with current filters/threshold. Showing raw FAISS matches (auto-fallback).")
        idxs2, scores2 = faiss_search_by_vector(qemb, k=top_k + 1)
        raw = []
        for j, s in zip(idxs2, scores2):
            j = int(j)
            if j == i:
                continue
            raw.append((j, float(s), 0.0, float(s)))
            if len(raw) >= top_k:
                break
        hits = raw

    return query, hits

# ----------------------------
# Tabs
# ----------------------------
tab1, tab2 = st.tabs(["🔎 Match Products", "📊 Analytics & Evaluation"])

with tab1:
    left, right = st.columns([1.05, 1.0], gap="large")

    with left:
        st.subheader("Search")

        if mode == "By Product ID":
            pid = st.text_input("Enter Product ID", placeholder="e.g., AND196")
            if pid:
                pid = pid.strip()
                if pid not in id_to_idx:
                    st.error("Product ID not found in dataset.")
                    st.stop()

                i = id_to_idx[pid]
                product_card(df, i, "Selected Product")

                query, hits = search_by_product_idx(i)
                st.caption(f"Query used for hybrid re-ranking: “{query}”")

                res_df = build_display_table(df, hits)
                st.markdown("### Top Matches")
                st.dataframe(res_df, use_container_width=True, height=420)

                st.download_button(
                    "⬇️ Download results CSV",
                    data=res_df.to_csv(index=False).encode("utf-8"),
                    file_name=f"matches_{pid}.csv",
                    mime="text/csv"
                )
        else:
            q = st.text_input("Enter search query", value="westernwear women")
            if q:
                hits = search_by_text(q)
                res_df = build_display_table(df, hits)
                st.markdown("### Top Matches")
                st.dataframe(res_df, use_container_width=True, height=420)

                st.download_button(
                    "⬇️ Download results CSV",
                    data=res_df.to_csv(index=False).encode("utf-8"),
                    file_name="matches_query.csv",
                    mime="text/csv"
                )

    with right:
        st.subheader("Compare (side-by-side)")
        st.caption("Paste two Product IDs to compare them side-by-side.")

        sel_pid = st.text_input("Selected Product ID", value="", placeholder="e.g., AND196")
        match_pid = st.text_input("Match Product ID", value="", placeholder="e.g., AND524")

        if sel_pid.strip() in id_to_idx and match_pid.strip() in id_to_idx:
            sel_idx = id_to_idx[sel_pid.strip()]
            m_idx = id_to_idx[match_pid.strip()]
            c1, c2 = st.columns(2, gap="large")
            with c1:
                product_card(df, sel_idx, "Selected")
            with c2:
                product_card(df, m_idx, "Matched")
        else:
            st.info("Enter valid Product IDs in both boxes to compare.")

with tab2:
    st.subheader("Dataset Overview")
    c1, c2, c3, c4 = st.columns(4)
    c1.metric("Total products", f"{len(df):,}")
    c2.metric("Unique categories", f"{df.get('Category', pd.Series(dtype=str)).nunique():,}")
    c3.metric("Unique brands", f"{df.get('BrandName', pd.Series(dtype=str)).nunique():,}")
    c4.metric("Filtered products", f"{len(allowed_idx):,}")

    cc1, cc2 = st.columns(2, gap="large")
    with cc1:
        st.markdown("#### Top Categories")
        if "Category" in df.columns:
            top_cat = df["Category"].astype(str).value_counts().head(15).reset_index()
            top_cat.columns = ["Category", "Count"]
            st.dataframe(top_cat, use_container_width=True, height=360)
        else:
            st.caption("No Category column found.")

    with cc2:
        st.markdown("#### Top Brands")
        if "BrandName" in df.columns:
            top_brand = df["BrandName"].astype(str).value_counts().head(15).reset_index()
            top_brand.columns = ["BrandName", "Count"]
            st.dataframe(top_brand, use_container_width=True, height=360)
        else:
            st.caption("No BrandName column found.")

    st.markdown("---")
    st.subheader("Evaluation Metrics")
    st.caption("Loaded from metrics.csv (generated in notebook).")
    if metrics is not None:
        st.dataframe(metrics, use_container_width=True)
    else:
        st.warning("metrics.csv not found in app_assets. Generate and save metrics in the notebook.")

    st.markdown("---")
    st.subheader("How this works")
    st.write(
        """
        - **Embeddings** convert each product text into a dense vector.
        - **FAISS** retrieves top candidates using cosine similarity (IndexFlatIP with normalized vectors).
        - **Hybrid re-ranking (optional)** mixes semantic similarity with a fuzzy keyword score for better practical matching.
        - **Filters** can restrict results by category/brand/price/discount, but missing values are allowed.
        """
    )


Overwriting /content/app.py


In [26]:
from pyngrok import ngrok
import subprocess

# start app
proc = subprocess.Popen(["streamlit", "run", "/content/app.py", "--server.port", "8501", "--server.address", "0.0.0.0"])

# connect
ngrok.set_auth_token("PASTE_YOUR_TOKEN_IN_COLAB")  # keep it private
public_url = ngrok.connect(8501)
print("Open:", public_url)


Open: NgrokTunnel: "https://pseudonymously-matrilateral-jenee.ngrok-free.dev" -> "http://localhost:8501"


In [23]:
df = pd.read_csv("/content/app_assets/products.csv")

print("Total rows:", len(df))
print("Empty text rows:", (df["Product Name"].isna() | (df["Product Name"].astype(str).str.strip()=="")).sum())
print("Sample names:")
print(df["Product Name"].head(10).tolist())


Total rows: 4566
Empty text rows: 0
Sample names:
['Cologne Fragrance', 'DRW1 - Westernwear-Women', 'DRW2 - Westernwear-Women', 'DRW3 - Westernwear-Women', 'DRW4 - Westernwear-Women', 'DRW5 - Westernwear-Women', 'DRW6 - Westernwear-Women', 'DRW7 - Westernwear-Women', 'DRW8 - Westernwear-Women', 'DRW9 - Westernwear-Women']


In [24]:
# raw faiss search (no filters)
q = "westernwear women"
qemb = embedder.encode([q], convert_to_numpy=True, normalize_embeddings=True).astype("float32")
scores, idxs = index.search(qemb, 10)

print("Query:", q)
for i, s in zip(idxs[0], scores[0]):
    r = df.iloc[int(i)]
    print(round(float(s),4), r["Product ID"], r["Product Name"], r["Category"], r["BrandName"])


Query: westernwear women
0.7264 AND196 AND196 - Westernwear-Women Westernwear-Women and
0.7016 AND524 AND524 - Westernwear-Women Westernwear-Women and
0.7013 AND465 AND465 - Westernwear-Women Westernwear-Women and
0.7009 AND364 AND364 - Westernwear-Women Westernwear-Women and
0.6994 AND84 AND84 - Westernwear-Women Westernwear-Women and
0.6969 AND228 AND228 - Westernwear-Women Westernwear-Women and
0.6967 AND4 AND4 - Westernwear-Women Westernwear-Women and
0.6951 COVER STORY137 COVER STORY137 - Westernwear-Women Westernwear-Women cover story
0.695 AND164 AND164 - Westernwear-Women Westernwear-Women and
0.6945 AND670 AND670 - Westernwear-Women Westernwear-Women and
